In [1]:
import numpy as np
import torch
from torchvision import transforms
from tqdm import tqdm


import PIL

import matplotlib.pyplot as plt


In [45]:
import open_clip
from open_clip import tokenizer
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion400m_e32')

In [ ]:
data = np.load('/cluster/scratch/nkoisheke/gen_images.npy')

In [ ]:
%matplotlib inline

temp_image = data[0]

plt.imshow(temp_image)

# Check on generated images

## Get image features

In [26]:
image_features = []
batch_size = 64

with torch.no_grad():
    for i in tqdm(range(0, len(data), batch_size)):
        batch = []
        for j in range(i, min(i+batch_size, len(data))):
            img = PIL.Image.fromarray(data[j])
            img = preprocess(img)
            batch.append(img)
        batch = torch.tensor(np.stack(batch))
        img_feats = model.encode_image(batch).float()
        image_features.append(img_feats)
image_features_torch = torch.concatenate(image_features)

100%|██████████| 59/59 [08:24<00:00,  8.55s/it]


 Clean the data

In [27]:
del data

## Get text features

In [28]:
# read csv file
import pandas as pd
df = pd.read_csv('/cluster/scratch/nkoisheke/gen_images.csv')
df.head()

,uid,artist,prompt,sd_prompt,sd_version
0,0,Vincent van Gogh,An exciting and adrenaline-fueled scene of a h...,An exciting and adrenaline-fueled scene of a h...,2.1
1,1,Vincent van Gogh,A spooky and eerie abandoned carnival scene wi...,A spooky and eerie abandoned carnival scene wi...,2.1
2,2,Vincent van Gogh,A mystical and spiritual scene of a meditating...,A mystical and spiritual scene of a meditating...,2.1
3,3,Vincent van Gogh,A dreamy and surreal cloudscape scene with flu...,A dreamy and surreal cloudscape scene with flu...,2.1
4,4,Vincent van Gogh,A dynamic and exciting skateboard park with ju...,A dynamic and exciting skateboard park with ju...,2.1


In [29]:
artists = df['artist'].unique()
text_features = []

with torch.no_grad():
    for artist in artists:
        prompt = "The following work is done in the style of " + artist
        text_tokens = tokenizer.tokenize(prompt)
        txt_feat = model.encode_text(text_tokens).float()
        text_features.append(txt_feat)
text_features_torch = torch.concatenate(text_features)

In [31]:
3735 / 15

249.0

In [32]:
idx2artist = {}
artist2idx = {}
for i, artist in enumerate(artists):
    idx2artist[i] = artist
    artist2idx[artist] = i

## Compute cosine similarity

In [33]:
image_features_torch /= image_features_torch.norm(dim=-1, keepdim=True)
text_features_torch /= text_features_torch.norm(dim=-1, keepdim=True)

text_probs = (100.0 * image_features_torch @ text_features_torch.T).softmax(dim=-1)
top_probs, top_k_labels = text_probs.cpu().topk(5, dim=-1)

In [34]:
gt_labels = torch.tensor([artist2idx[x] for x in df['artist'].to_list()])

top_one_labels = top_k_labels[:, 0]

print(gt_labels.shape, top_one_labels.shape)

correct = (gt_labels == top_one_labels).sum()
print(f"Top 1 score is {round((correct / gt_labels.shape[0]).item() * 100, 2)}")


topk_correct = 0
for i in range(5):
    top_one_labels = top_k_labels[:, i]
    correct = (gt_labels == top_one_labels).sum()
    topk_correct += correct
print(f"Top 5 score is {round((topk_correct / gt_labels.shape[0]).item() * 100, 2)}")
# correct = (gt_labels.repeat() == top_k_labels).sum()
# print(correct)

torch.Size([3735]) torch.Size([3735])
Top 1 score is 4.58
Top 5 score is 12.64


# Check on real images

In [35]:
import os
path = "/cluster/scratch/nkoisheke/artworks_images/"
df = pd.read_csv('../Scraping/googleart/Data/googleart_art_data_final.csv', sep='\t')

In [36]:
df.head()

,artist_name,artwork_title,artwork_date,artwork_url,image_url,image_path
0,Vincent van Gogh,Self-Portrait,1889,https://artsandculture.google.com/asset/self-p...,blob:https://artsandculture.google.com/46b00d1...,./Scraping/googleart/Data/artworks_images/imag...
1,Vincent van Gogh,The Starry Night,1889,https://artsandculture.google.com/asset/the-st...,blob:https://artsandculture.google.com/30b15a6...,./Scraping/googleart/Data/artworks_images/imag...
2,Vincent van Gogh,Almond Blossom,February 1890 - 1890,https://artsandculture.google.com/asset/almond...,blob:https://artsandculture.google.com/78cc161...,./Scraping/googleart/Data/artworks_images/imag...
3,Vincent van Gogh,Country road in Provence by night,c. 12 - 15 May 1890,https://artsandculture.google.com/asset/countr...,blob:https://artsandculture.google.com/4873671...,./Scraping/googleart/Data/artworks_images/imag...
4,Vincent van Gogh,Starry Night,1888,https://artsandculture.google.com/asset/starry...,blob:https://artsandculture.google.com/fe464bc...,./Scraping/googleart/Data/artworks_images/imag...


In [37]:
image_paths = [path + img.split('/')[-1] for img in df["image_path"].to_list()]

## Get image features

In [38]:
image_features = []
batch_size = 64

with torch.no_grad():
    for i in tqdm(range(0, len(image_paths), batch_size)):
        batch = []
        for j in range(i, min(i+batch_size, len(image_paths))):
            img = PIL.Image.open(image_paths[j])
            img = preprocess(img)
            batch.append(img)
        batch = torch.tensor(np.stack(batch))
        img_feats = model.encode_image(batch).float()
        image_features.append(img_feats)
image_features_torch = torch.concatenate(image_features)

100%|██████████| 19/19 [02:43<00:00,  8.58s/it]


In [39]:
artists = df['artist_name'].unique()
text_features = []

with torch.no_grad():
    for artist in artists:
        prompt = "The following work is done in the style of " + artist
        text_tokens = tokenizer.tokenize(prompt)
        txt_feat = model.encode_text(text_tokens).float()
        text_features.append(txt_feat)
text_features_torch = torch.concatenate(text_features)

## Compute probabilities

In [40]:
image_features_torch /= image_features_torch.norm(dim=-1, keepdim=True)
text_features_torch /= text_features_torch.norm(dim=-1, keepdim=True)

text_probs = (100.0 * image_features_torch @ text_features_torch.T).softmax(dim=-1)
top_probs, top_k_labels = text_probs.cpu().topk(5, dim=-1)

In [41]:
gt_labels = torch.tensor([artist2idx[x] for x in df['artist_name'].to_list()])

top_one_labels = top_k_labels[:, 0]

print(gt_labels.shape, top_one_labels.shape)

correct = (gt_labels == top_one_labels).sum()
print(f"Top 1 score is {round((correct / gt_labels.shape[0]).item() * 100, 2)}")


topk_correct = 0
for i in range(5):
    top_one_labels = top_k_labels[:, i]
    correct = (gt_labels == top_one_labels).sum()
    topk_correct += correct
print(f"Top 5 score is {round((topk_correct / gt_labels.shape[0]).item() * 100, 2)}")
# correct = (gt_labels.repeat() == top_k_labels).sum()
# print(correct)

torch.Size([1181]) torch.Size([1181])
Top 1 score is 45.89
Top 5 score is 67.49
